## Spark Streaming Notebook

This notebook consumes a message from a kafka topic, writes the message into another kafka topic.

#### Create a new Spark Session, and add a kafka package from a jar file

In [1]:
from pyspark.sql import SparkSession

# Spark session and context
spark = (SparkSession
        .builder
        .master('local')
        .appName('kafka-streaming')
        # Add kafka package
        .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.11:2.4.5")
        .getOrCreate())

sc = spark.sparkContext

#### Now we have to read the message from the kafka stream

Create a new instance of the `readStream` object, add kafka bootstrap servers with appropriate port number to internally connect to kafka, and then subscribe to the ingestion topic. Use `selectExpr` to format the messages we're getting from Kafka.

In [2]:
df = spark.readStream.format("kafka") \
    .option("kafka.bootstrap.servers", "kafka:9092") \
    .option("subscribe", "ingestiontopic") \
    .option("failOnDataLoss", "false") \
    .load()

df.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)")

DataFrame[key: string, value: string]

#### Create a small temporary view for SparkSQL

Using Spark SQL, from the dataframe created by the readStream, create a new Temporary view.

Then, write out the message to the console of the environment with append mode.

In [3]:
df.createOrReplaceTempView("message")

res = spark.sql("SELECT * FROM message")
res.writeStream.format("console") \
        .outputMode("append") \
        .start()

#### Write message back into kafka in another topic that you are going to listen to with a local consumer

use `/tmp` as the checkpoint location within the docker container

In [4]:
ds = df \
    .writeStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "kafka:9092") \
    .option("topic", "sparkoutput") \
    .option("checkpointLocation", "/tmp") \
    .option("failOnDataLoss", "false") \
    .start() \
    .awaitTermination()

StreamingQueryException: 'Multiple streaming queries are concurrently using file:/tmp/offsets/192\n=== Streaming Query ===\nIdentifier: [id = 389536d5-10a1-4250-8509-95b00c613ce2, runId = a1521fe7-2de3-4488-aab3-6eeaf0fe0973]\nCurrent Committed Offsets: {KafkaV2[Subscribe[ingestiontopic]]: {"ingestiontopic":{"0":311}}}\nCurrent Available Offsets: {KafkaV2[Subscribe[ingestiontopic]]: {"ingestiontopic":{"0":312}}}\n\nCurrent State: ACTIVE\nThread State: RUNNABLE\n\nLogical Plan:\nKafkaV2[Subscribe[ingestiontopic]]'